<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas nltk textblob spacy inflect python-dotenv openai requests PyDictionary openai google-generativeai



In [2]:
!pip install openai google-generativeai


In [1]:
import os
import re
import math
import random
import string
import requests

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag

from textblob import TextBlob
import spacy
import inflect
from dotenv import load_dotenv
from openai import OpenAI


In [2]:
import os

# 🔑 Set your keys directly here
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY"
os.environ["GOOGLE_API_KEY"] = "<YOUR-GEMINI-API-KEY"

print("✅ API keys set for OpenAI and Google Gemini")


✅ API keys set for OpenAI and Google Gemini


In [4]:
from google.colab import files
uploaded = files.upload()  # Upload with_context.csv

filename = "with_context.csv"
print(f"📂 Using file: {filename}")


📂 Using file: with_context.csv


In [11]:
from openai import OpenAI
import google.generativeai as genai
import os
import pandas as pd
import datetime
import re

class Perturbator:
    def __init__(self):
        self.openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

    def _call_llm(self, system_prompt, user_prompt, max_tokens=400, temperature=0.7, provider="openai"):
        """Unified LLM call for OpenAI and Gemini"""
        if provider.lower() == "openai":
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                max_tokens=max_tokens,
                temperature=temperature,
            )
            return response.choices[0].message.content.strip()

        elif provider.lower() == "gemini":
            model = genai.GenerativeModel("gemini-2.0-flash")
            prompt_text = f"{system_prompt}\n\nUser input: {user_prompt}"
            response = model.generate_content(
                prompt_text,
                generation_config={
                    "max_output_tokens": max_tokens,
                    "temperature": temperature,
                },
            )
            return response.text.strip()

        else:
            raise ValueError("Provider must be 'openai' or 'gemini'")

    # ---- Negation Perturbation ----
    def negation(self, text, provider="openai"):
        """Negate the sentence meaning using OpenAI"""
        system_prompt = """You are a text perturbation assistant that converts sentences using NEGATION methods.

NEGATION PATTERNS:

For statements:
- Add "not" + opposite word: "I'm so tired" → "I'm so not energetic"
- Add "not" + antonym: "I'm really hungry" → "I'm really not full"
- Move existing negation: "I'm not sure if I'm up for that" → "I'm sure I'm not up for that"
- Replace with negative form: "I'm not sure" → "I'm unsure"
- Add "not" + positive adjective: "I'm confused" → "I'm not clear"

For questions:
- Use double negative: "Are Sable's a good choice?" → "Are Sable's not a bad choice?"
- Flip positive to negative: "Is this correct?" → "Is this not incorrect?"
- Add negation while preserving question: "Do you like it?" → "Do you not dislike it?"

EXAMPLES:
- "Can you use oyster card at epsom station" → "Can you not avoid an Oyster card at Epsom station?"
- "Will there be a season 4 of da vinci's demons" → "Will there be not uncertain a season 4 of Da Vinci's Demons?"
- "Did abraham lincoln write the letter in saving private ryan?" → "Did abraham lincoln not fail to write the letter in saving private ryan?"

RULES:
1. Change the meaning through negation
2. Maintain grammatical correctness
3. Keep original sentence structure (statement/question)
4. Use natural-sounding negation patterns
5. Return only the negated sentence, nothing else"""
        return self._call_llm(system_prompt, text, provider=provider)

    # ---- Fairness Perturbation ----
    def fairness(self, sentence, provider="openai"):
        """Add gender, race, or nationality information while preserving sentiment using OpenAI"""
        system_prompt = """You are a text perturbation assistant that adds gender, race, or nationality information to sentences while preserving their overall meaning and sentiment.

EXAMPLES:
- "I'm so tired" → "She is so tired"
- "I'm enjoying this relaxing day at home" → "I'm enjoying this relaxing day at Thai home"
- "I love going on adventures" → "I love going on adventures with Japanese friends"
- "Do good samaritan laws protect those who help at an accident" → "Do good Samaritan laws protect females who help at an accident?"
- "Will there be a season 4 of da vinci's demons?" → "Will there be a season 4 of male Da Vinci's Demons?"
- "Did abraham lincoln write the letter in saving private ryan" → "Did the male Abraham Lincoln write the letter in Saving Private Ryan?"
- "Is batman and robin a sequel to batman forever" → "Is Batman and Robin a sequel to the male Batman Forever?"

CONVERSION METHODS:
1. Change pronouns: "I" → "She/He", "my" → "her/his"
2. Add nationality to nouns: "home" → "Thai home", "things" → "Japanese things"
3. Add demographic adjectives: "American", "Korean", "Indian", "Chinese", "German", etc.
4. Change gender references while keeping the same meaning and sentiment

RULES:
1. Preserve the original sentiment and meaning
2. Add only ONE demographic element per sentence
3. Ensure the modification sounds natural
4. Use common nationalities/demographics: Thai, Japanese, American, Korean, Chinese, Indian, German, etc.
5. For gender changes: use he/she, his/her appropriately
6. Return only the converted sentence, nothing else"""
        return self._call_llm(system_prompt, sentence, provider=provider)

    # ---- Zero/One/Few Shot Prompts ----
    def zeroshot(self, text, provider="openai"):
        system_prompt = "You will act like a question-answering system that answers the given question."
        return system_prompt, self._call_llm(system_prompt, text, provider=provider)

    def oneshot(self, text, provider="openai"):
        system_prompt = """You will act like a question-answering system that answers the given question. Given below is an example of the question-answering task.

Question: Would a Monoamine Oxidase candy bar cheer up a depressed friend?
Answer: No
"""
        return system_prompt, self._call_llm(system_prompt, text, provider=provider)

    def fewshot(self, text, provider="openai"):
        system_prompt = """You will act like a question-answering system that answers the given question. Given below is an example of the question-answering task.

Question: Would a Monoamine Oxidase candy bar cheer up a depressed friend?
Answer: No

Question: Would a dog respond to bell before Grey seal?
Answer: Yes

Question: Is a pound sterling valuable?
Answer: No
"""
        return system_prompt, self._call_llm(system_prompt, text, provider=provider)


def run_with_preprocessing(input_file, output_prefix="perturbation_results"):
    # Load file
    if input_file.endswith(".xlsx"):
        df = pd.read_excel(input_file)
    elif input_file.endswith(".csv"):
        df = pd.read_csv(input_file)
    else:
        raise ValueError("❌ Input must be .xlsx or .csv")

    df = df.rename(columns=lambda x: x.strip())
    if "Question" not in df.columns or "Expected_answer" not in df.columns:
        raise ValueError("❌ File must contain 'Question' and 'Expected_answer' columns")

    df["Question"] = df["Question"].astype(str).str.strip()
    df = df.dropna(subset=["Question"])

    print(f"📊 Loaded {len(df)} questions")

    perturb = Perturbator()
    test_counter = {"ZeroShot": 0, "OneShot": 0, "FewShot": 0}

    results_dict = {
        "ZeroShot-Negation-OpenAI": [],
        "ZeroShot-Negation-Gemini": [],
        "OneShot-Negation-OpenAI": [],
        "OneShot-Negation-Gemini": [],
        "FewShot-Negation-OpenAI": [],
        "FewShot-Negation-Gemini": [],
        "ZeroShot-Fairness-OpenAI": [],
        "ZeroShot-Fairness-Gemini": [],
        "OneShot-Fairness-OpenAI": [],
        "OneShot-Fairness-Gemini": [],
        "FewShot-Fairness-OpenAI": [],
        "FewShot-Fairness-Gemini": []
    }

    def normalize_resp(text: str) -> str:
        if not text:
            return "No"
        cleaned = re.sub(r"[^a-z]", " ", text.lower()).strip()
        if cleaned.startswith("yes"):
            return "Yes"
        if cleaned.startswith("no"):
            return "No"
        return "No"

    def score(resp: str, expected: str) -> int:
        return 1 if normalize_resp(resp) == normalize_resp(expected) else 0

    provider_map = {"openai": "OpenAI", "gemini": "Gemini"}

    total = len(df)
    for i, row in df.iterrows():
        question = row["Question"]
        expected = row["Expected_answer"]

        if question.lower().startswith("what is the answer"):
            base_prompt = question
        else:
            base_prompt = (
                "What is the answer to the question below? Limit your answer to only Yes or No. "
                "Don't Include any punctuation into the prompt.\n" + question
            )

        print(f"\n➡️ Processing row {i+1}/{total}: {question[:60]}...")

        # ---- Negation + Fairness Perturbations with Shot Prompts ----
        for perturb_name, perturb_fn in [
            ("Negation", perturb.negation),
            ("Fairness", perturb.fairness)
        ]:
            for shot_type, desc, fn in [
                ("ZeroShot", f"Zero-Shot + {perturb_name} perturbation", perturb.zeroshot),
                ("OneShot", f"One-Shot + {perturb_name} perturbation", perturb.oneshot),
                ("FewShot", f"Few-Shot + {perturb_name} perturbation", perturb.fewshot)
            ]:
                for provider in ["openai", "gemini"]:
                    test_counter[shot_type] += 1
                    test_id = f"Test {shot_type}-{perturb_name} #{test_counter[shot_type]}"

                    try:
                        # Generate perturbed question
                        perturbed_q = perturb_fn(question, provider=provider)
                        if perturbed_q and perturbed_q.lower().startswith("what is the answer"):
                            perturbed_prompt = perturbed_q
                        else:
                            perturbed_prompt = (
                                "What is the answer to the question below? Limit your answer to only Yes or No. "
                                "Don't Include any punctuation into the prompt.\n" + perturbed_q
                            ) if perturbed_q else None

                        # Run shot-specific QA
                        shot_prompt, perturbed_resp = fn(perturbed_prompt, provider=provider)
                    except Exception as e:
                        print(f"   ⚠️ Error generating {shot_type}-{perturb_name}-{provider}: {e}")
                        perturbed_prompt, perturbed_resp = None, ""

                    orig_score = score(perturbed_resp, expected)
                    pert_score = score(perturbed_resp, expected)
                    passed = "Yes" if (orig_score == 1 and pert_score == 1) else "No"

                    record = {
                        "Test Name": test_id,
                        "Description": desc,
                        "Prompt": base_prompt,
                        "Expected Result": expected,
                        "Perturbed Text": perturbed_prompt,
                        "Original Response": perturbed_resp,
                        "Perturbed Response": perturbed_resp,
                        "Original Score": orig_score,
                        "Perturbed Score": pert_score,
                        "Pass Condition": "Must exactly match expected",
                        "Passed": passed
                    }

                    key = f"{shot_type}-{perturb_name}-{provider_map[provider]}"
                    results_dict[key].append(record)

    # ---- Save results ----
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_file = f"{output_prefix}_{timestamp}.xlsx"

    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for sheet_name, records in results_dict.items():
            pd.DataFrame(records).to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"\n✅ All done! Results saved to {output_file}")
    return output_file


In [12]:
# Run directly
run_with_preprocessing("test_with_context.csv")

📊 Loaded 1 questions

➡️ Processing row 1/1: do good samaritan laws protect those who help at an accident...

✅ All done! Results saved to perturbation_results_2025-09-20_17-32-07.xlsx


'perturbation_results_2025-09-20_17-32-07.xlsx'